# CRACK DETECTION using GAPS DATASET

## IMPORT LIBRARIES

In [11]:
from __future__ import print_function
#import readdata
from tensorflow import keras
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import os

## LOAD DATA

# v2 6-classes 256

In [5]:
from importlib.machinery import SourceFileLoader
MODULENAME = "loadgaps"
MODULEPATH = "/home/ali/my_project/large_files/gaps/loadgaps.py"
lgaps = SourceFileLoader(MODULENAME, MODULEPATH).load_module()

x_train, y_train, x_valid, y_valid, x_test, y_test = lgaps.loadv2_ZEB256('low')

in the gaps v2 ZEB256
max tr = 25, max va = 5, max te = 5
low-load data in process
current: tr:2, va:1, te:1
loading 1 chunk  out of 2 of training in v2 gaps
loading 2 chunk  out of 2 of training in v2 gaps
loading 1 chunk  out of 1 of valid in v2 gaps
loading 1 chunk  out of 1 of test in v2 gaps
data name  is: gaps v2 ZEB256
input shape is : (256, 256, 3)
train num is : 4000
valid num is : 2000
test  num is : 2000
number of classes are : 6


# v1 binary 64

In [ ]:
from importlib.machinery import SourceFileLoader
MODULENAME = "loadgaps"
MODULEPATH = "/home/ali/my_project/large_files/gaps/loadgaps.py"
lgaps = SourceFileLoader(MODULENAME, MODULEPATH).load_module()
x_train, y_train, x_valid, y_valid, x_test, y_test = lgaps.loadv1_NVD64() 

## BUILD MODEL

In [6]:
model_archive = {
                '1' : '2CCP_1FDD',
                '2' : 'DUPLICATE',
                '3' : 'Transfer_learning_vgg'
                    }
print(model_archive)

{'1': '2CCP_1FDD', '2': 'DUPLICATE', '3': 'Transfer_learning_vgg'}


In [9]:
print(input_shape)
print(type(input_shape))

print(num_classes)

(256, 256, 3)
<class 'tuple'>
6


In [ ]:
(256, 256, 3)
6

In [10]:
import readmodel
model_name = 'model_1' # 'model_1', 'model_2', or model_3
input_shape = (256, 256, 3)
num_classes = 6
model = readmodel.modelchoose(model_name,input_shape, num_classes)
model.summary()

last change
X current model is: model_1_2CCP_1FDD
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_7 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv

In [7]:
import readmodel
model_name = 'model_1' # 'model_1', 'model_2', or model_3
input_shape = x_train[0,:,:,:].shape
num_classes = y_train.shape[1]
model = readmodel.modelchoose(model_name,input_shape, num_classes)
model.summary()

last change
X current model is: model_1_2CCP_1FDD
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d

## COMPILE MODEL

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.01, decay=1e-6)

print(num_classes)
# Let's train the model using RMSprop
if num_classes == 2:
    loss = 'binary_crossentropy'
if num_classes == 6:
    loss = 'categorical_crossentropy'
    
print(loss)

model.compile( loss = loss, # loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

 ## GENERATE IMAGES

In [ ]:
# create an instance of ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)


# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

## HYPERPARAMETERS

In [ ]:
epochs = 5 # 200
batch_size = 50
#num_classes = y_train_binary.shape[1]


## FIT DATA TO MODEL using IMG GENERATOR

In [ ]:
# fits the model on batches with real-time data augmentation:
history = model.fit_generator(datagen.flow(x_train, y_train_binary, batch_size=batch_size),
                    epochs=epochs, 
                    steps_per_epoch=len(x_train) / batch_size,
                    validation_data=(x_valid, y_valid_binary),
                    shuffle=True)
                    

## FIT DATA to MODEL w/o IMG GENRATOR

In [ ]:
history_2 = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_valid, y_valid),
              shuffle=True)

## MODEL SCORE

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

## ACCURACY & VAL vs EPOCHS PLOTS

In [ ]:
history = history_2

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

## SAVE MODEL and WEIGHTS

In [ ]:
model_weight_name = model_name + f'_epochs_{epochs}_batchsize_{batch_size}.h5'
print(model_weight_name)

In [ ]:
os.getcwd()

In [ ]:
# Save model and weights
#model_name = 'gaps_model_1.h5'
model_weight_name = model_name + f'_epochs_{epochs}_batchsize_{batch_size}.h5'

#os.getcwd()
# save_dir = os.path.join(os.getcwd(), 'saved_models')
save_dir = '/home/ali/my_project/gaps/saved_models'
model_path = os.path.join(save_dir, model_weight_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# GRID SEARCH

## GRID SEARCH FIT for H_PARAM TUNNING

In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_model(model_name):
    #model_name = 'model_3' # 'model_1', 'model_2', 'model_3'
    model = readmodel.modelchoose(model_name,input_shape, num_classes)

    model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])
    
    return model

modelC = KerasClassifier(build_fn=create_model, verbose=2)

# using grid search
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.01, decay=1e-6)

epochs = [2]
model_name = ['model_1', 'model_3']

param_grid = dict(model_name = model_name)

grid = GridSearchCV(estimator = modelC , param_grid=param_grid)
grid_result = grid.fit(x_train, y_train_binary)

## GRID SEARCH RESULTS

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
history = grid_result.best_estimator_.model.history

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label = 'loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
history.history